## Примерный подход к решению кейса
1. Обработка описаний поручений:

- Создать систему для объединения синонимичных формулировок. Можно использовать алгоритмы Word2Vec, TF-IDF для классификации описаний, объединяя задания с похожими требованиями (например, «каменный дракон» и «ходячая гора»).
- Выделить общие типы поручений с использованием алгоритмов (например, K-means) для  группировки схожих поручений, что упростит процесс их назначения подходящим командам.

2. Анализ навыков героев:
- Создать профиль каждого героя на основе его опыта, навыков и отзывов. Использовать данные из дневников для определения его сильных и слабых сторон. Например, если герой часто выполняет задания, связанные с определенным типом монстра, это может быть учтено при назначении его на аналогичные поручения. Присваивать вес каждому навыку героя, учитывая скорость выполнения задания и оценки от заказчиков. Чем выше оценка и скорость выполнения, тем более опытен герой в этом направлении.

3. Формирование команд:
- Задача Аркаши – находить баланс между командной численностью и эффективностью. Это можно решить с помощью алгоритма оптимизации, который будет назначать оптимальные команды для максимизации общей выручки.
- Использовать модель машинного обучения для предсказания вероятности успеха команды. Модель может учитывать сложность задания, навыки героев, предыдущий опыт, оценки и затраченное время. 

Создадим UML-диаграмму для визуализации таблиц и их зависимостей.



In [1]:
# Cкачиваем необходимые библиотеки для работы
#!pip install pandas numpy matplotlib seaborn sentence_transformers

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Записываем исходные данные в датафреймы
cases = pd.read_csv('cases.csv', sep=';')
diaries = pd.read_csv('diaries.csv', sep=';')
marks = pd.read_csv('marks.csv', sep=';')

In [4]:
# Выводим столбцы датафреймов
print(cases.columns)
print(diaries.columns)
print(marks.columns)

Index(['﻿Номер поручения', 'Заказчик', 'Дата поручения', 'Выполнено',
       'Дата выполнения', 'Затрачено дней', 'Сумма вознаграждения',
       'Описание'],
      dtype='object')
Index(['Номер поручения', 'Герой', 'Запись в дневнике', 'Затрачено часов',
       'Роль'],
      dtype='object')
Index(['Номер поручения', 'Герой', 'Оценка за качество', 'Оценка по срокам',
       'Оценка за вежливость'],
      dtype='object')


In [5]:
# Проверям датафрейм на наличие NaN (не упускаем из внимания, что это просто могут быть невыполненные поручения)
print(cases.isna().sum(),'\n')
print(marks.isna().sum(),'\n')
print(diaries.isna().sum())

Номер поручения         0
Заказчик                 0
Дата поручения           0
Выполнено                0
Дата выполнения         19
Затрачено дней          19
Сумма вознаграждения     0
Описание                 0
dtype: int64 

Номер поручения         0
Герой                   0
Оценка за качество      0
Оценка по срокам        0
Оценка за вежливость    0
dtype: int64 

Номер поручения      0
Герой                0
Запись в дневнике    0
Затрачено часов      0
Роль                 0
dtype: int64


In [6]:
# Выведем невыполненные поручения
cases[cases['Выполнено'] == 'нет']

,﻿Номер поручения,Заказчик,Дата поручения,Выполнено,Дата выполнения,Затрачено дней,Сумма вознаграждения,Описание
56,11056,Мария,1053-09-06,нет,NaN,NaN,23500,По дороге из деревни у меня пропала драгоценно...
134,11134,Иван,1053-09-04,нет,NaN,NaN,27500,В пещере появвилось огромное каменное чудовище...
143,11143,Егор,1053-10-18,нет,NaN,NaN,10500,В пещере завёлся дракон. Нужно его убить. Это ...
161,11161,Эмилио,1053-10-03,нет,NaN,NaN,7500,В городе у меня потерялся рюкзак. Нужно найти ...
218,11218,Олег,1053-10-01,нет,NaN,NaN,16000,Недалеко от города монстры похитили путников. ...
232,11232,Бабушка Синь,1053-10-22,нет,NaN,NaN,17500,В деревне монстры похитили путников. Нужно спа...
234,11234,Олег,1053-10-24,нет,NaN,NaN,17000,В лесу по дороге от пещеры заметили разбойнико...
285,11285,Чарли,1053-09-30,нет,NaN,NaN,5000,В городе у меня потерялся рюкзак. Нужно найти ...
306,11306,Олег,1053-11-15,нет,NaN,NaN,17000,В деревне монстры похитили путников. Нужно спа...
310,11310,Надя,1053-09-24,нет,NaN,NaN,5000,По дороге из деревни монстры похитили путников...


In [7]:
# Удостоверимся, что по невыполненным поручениям нет записей в дневниках и оценок
print(diaries[diaries['Номер поручения'].isin(cases[cases['Выполнено'] == 'нет'])])
print(marks[marks['Номер поручения'].isin(cases[cases['Выполнено'] == 'нет'])])

Empty DataFrame
Columns: [Номер поручения, Герой, Запись в дневнике, Затрачено часов, Роль]
Index: []
Empty DataFrame
Columns: [Номер поручения, Герой, Оценка за качество, Оценка по срокам, Оценка за вежливость]
Index: []


In [8]:
#!pip install -U sentence-transformers

### Обработка описаний поручений

In [9]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer

task_descriptions = cases['Описание']

# Создание эмбеддингов с использованием Sentence-BERT
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings = model.encode(task_descriptions)

# Определение диапазона кластеров
# Максимальное количество кластеров не должно превышать (n_samples - 1)
n_samples = len(task_descriptions)

max_clusters = 150
range_n_clusters = list(range(2, max_clusters + 1))  # 2 до max_clusters включительно

# Листы для хранения результатов
inertia = []

# Перебираем различные количества кластеров и считаем метрики
for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(embeddings)
    
    # Метод локтя: инерция
    inertia.append(kmeans.inertia_)
    
# Создание графика
fig, ax1 = plt.subplots(1, 1, figsize=(10, 6))

# Метод локтя
ax1.plot(range_n_clusters, inertia, 'o-', color='blue')
ax1.set_title("Метод локтя")
ax1.set_xlabel("Количество кластеров", fontsize=12)
ax1.set_ylabel("Инерция (Сумма квадратов расстояний до центроидов)", fontsize=10)
ax1.grid(True)


# Настройка диапазона осей и увеличение шрифта
ax1.tick_params(axis='x', rotation=45, labelsize=10)
ax1.tick_params(axis='y', labelsize=10)
ax1.set_xlim(1, 150)

plt.tight_layout()
plt.show()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

Исходя из сгиба графика, оптимальное количество кластеров примерно равно 18.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import nltk
from nltk.corpus import stopwords

# загрузка стоп-слов
nltk.download('stopwords')
russian_stopwords = stopwords.words('russian')

# Преобразование текстов в числовые векторы с использованием TF-IDF
vectorizer = TfidfVectorizer(stop_words=russian_stopwords)  
X = vectorizer.fit_transform(task_descriptions)

n_clusters = 18

# Выполнение кластеризации с заданным количеством кластеров
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(X)

# Группировка заданий по кластерам
clustered_tasks = {}
for i, label in enumerate(cluster_labels):
    if label not in clustered_tasks:
        clustered_tasks[label] = []
    clustered_tasks[label].append(task_descriptions[i])

# Вывод группированных заданий по порядку кластеров
for cluster in sorted(clustered_tasks.keys()):
    print(f"Кластер {cluster}:")
    for task in clustered_tasks[cluster]:
        print(f"  - {task}")

Кластер 0:
  - Недалеко от города у меня был украден рюкзак. Нужно вернуть его как можно скорее.
  - В городе у меня был украден рюкзак. Нужно вернуть его как можно скорее.
  - Недалеко от города у меня был украден рюкзак. Нужно вернуть его как можно скорее.
  - Недалеко от города у меня был украден рюкзак. Нужно найти его как можно скорее.
  - В деревне у меня был украден рюкзак. Нужно найти его как можно скорее.
  - Недалеко от города у меня был украден рюкзак. Нужно вернуть его как можно скорее.
  - По дороге из деревни у меня был украден рюкзак. Нужно вернуть его как можно скорее.
  - В деревне у меня был украден рюкзак. Нужно найти его как можно скорее. Проверьте все места, где он мог быть оставлен.
  - Недалеко от города у меня был украден рюкзак. Нужно вернуть его как можно скорее. Проверьте все места, где он мог быть оставлен.
  - По дороге из деревни у меня был украден рюкзак. Нужно вернуть его как можно скорее.
  - Недалеко от города у меня был украден рюкзак. Нужно вернуть е

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vgrya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Анализ навыков героев